In [66]:
import pandas as pd
import numpy as np
data=pd.read_csv("/content/cancer_data.csv")
print(data.head())

         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38           77.58      386.1   
4  84358402         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   ...  radius_worst  texture_worst  perimeter_worst  area_wor

In [67]:
#data information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [68]:
data.isnull().sum()

,0
id,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0


In [69]:
#duplication
duplicate_rows=data[data.duplicated()]
print(duplicate_rows)

Empty DataFrame
Columns: [id, diagnosis, radius_mean, texture_mean, perimeter_mean, area_mean, smoothness_mean, compactness_mean, concavity_mean, concave points_mean, symmetry_mean, fractal_dimension_mean, radius_se, texture_se, perimeter_se, area_se, smoothness_se, compactness_se, concavity_se, concave points_se, symmetry_se, fractal_dimension_se, radius_worst, texture_worst, perimeter_worst, area_worst, smoothness_worst, compactness_worst, concavity_worst, concave points_worst, symmetry_worst, fractal_dimension_worst]
Index: []

[0 rows x 32 columns]


In [70]:
data_cleaned=data.drop_duplicates()

In [71]:
#Remove outlayout(z_score)

from scipy.stats import zscore

numeric_cols = data_cleaned.select_dtypes(include=[np.number]).columns
z_scores = zscore(data[numeric_cols])
z_scores_df = pd.DataFrame(z_scores, columns=numeric_cols)
outliers = (np.abs(z_scores_df) > 3).any(axis=1)
data_cleaned = data[~outliers]
data_cleaned.reset_index(drop=True, inplace=True)

print(data_cleaned.head())
print("Original shape:", data.shape)
print("Cleaned shape:", data_cleaned.shape)

         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842517         M        20.57         17.77          132.90     1326.0   
1  84300903         M        19.69         21.25          130.00     1203.0   
2  84358402         M        20.29         14.34          135.10     1297.0   
3    843786         M        12.45         15.70           82.57      477.1   
4    844359         M        18.25         19.98          119.60     1040.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.08474           0.07864          0.0869              0.07017   
1          0.10960           0.15990          0.1974              0.12790   
2          0.10030           0.13280          0.1980              0.10430   
3          0.12780           0.17000          0.1578              0.08089   
4          0.09463           0.10900          0.1127              0.07400   

   ...  radius_worst  texture_worst  perimeter_worst  area_wor

In [72]:
#label_encoding
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data_cleaned.loc[:,"diagnosis"] = le.fit_transform(data_cleaned["diagnosis"])
# Ensure the diagnosis column is of integer type
data_cleaned["diagnosis"] = data_cleaned["diagnosis"].astype(int)

/tmp/ipython-input-3239426513.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned["diagnosis"] = data_cleaned["diagnosis"].astype(int)


In [73]:
x=data_cleaned.drop("diagnosis",axis=1)
y=data_cleaned["diagnosis"]

In [74]:
#anova
from sklearn.feature_selection import SelectKBest, f_classif
selector=SelectKBest(score_func=f_classif, k=10)
selector.fit(x,y)

selected_features=x.columns[selector.get_support()]
x_anova=x[selected_features]
x_anova=pd.DataFrame(x_anova)

print("selected top features:")
print(x_anova)

selected top features:
     radius_mean  perimeter_mean  area_mean  concavity_mean  \
0          20.57          132.90     1326.0         0.08690   
1          19.69          130.00     1203.0         0.19740   
2          20.29          135.10     1297.0         0.19800   
3          12.45           82.57      477.1         0.15780   
4          18.25          119.60     1040.0         0.11270   
..           ...             ...        ...             ...   
482        14.05           91.38      600.4         0.04462   
483        20.92          143.00     1347.0         0.31740   
484        21.56          142.00     1479.0         0.24390   
485        20.13          131.20     1261.0         0.14400   
486        16.60          108.30      858.1         0.09251   

     concave points_mean  area_se  radius_worst  perimeter_worst  area_worst  \
0                0.07017    74.08         24.99            158.8      1956.0   
1                0.12790    94.03         23.57            1

In [75]:
print(x_anova.columns)

Index(['radius_mean', 'perimeter_mean', 'area_mean', 'concavity_mean',
       'concave points_mean', 'area_se', 'radius_worst', 'perimeter_worst',
       'area_worst', 'concave points_worst'],
      dtype='object')


In [76]:
#join the column
new_data=pd.concat([y,x_anova],axis=1)
print(new_data.head())
print(new_data.shape)

   diagnosis  radius_mean  perimeter_mean  area_mean  concavity_mean  \
0          1        20.57          132.90     1326.0          0.0869   
1          1        19.69          130.00     1203.0          0.1974   
2          1        20.29          135.10     1297.0          0.1980   
3          1        12.45           82.57      477.1          0.1578   
4          1        18.25          119.60     1040.0          0.1127   

   concave points_mean  area_se  radius_worst  perimeter_worst  area_worst  \
0              0.07017    74.08         24.99            158.8      1956.0   
1              0.12790    94.03         23.57            152.5      1709.0   
2              0.10430    94.44         22.54            152.2      1575.0   
3              0.08089    27.19         15.47            103.4       741.6   
4              0.07400    53.91         22.88            153.2      1606.0   

   concave points_worst  
0                0.1860  
1                0.2430  
2                0.1

In [77]:
#skew normalization
from sklearn.preprocessing import PowerTransformer

pt=PowerTransformer()
cols=['radius_mean', 'perimeter_mean', 'area_mean', 'concavity_mean',
       'concave points_mean', 'area_se', 'radius_worst', 'perimeter_worst',
       'area_worst', 'concave points_worst']
new_data[cols]=pt.fit_transform(new_data[cols])
skew_values=new_data.skew()
print(skew_values)
print(new_data.head())

diagnosis               0.732358
radius_mean             0.003733
perimeter_mean          0.003104
area_mean               0.001810
concavity_mean          0.207308
concave points_mean     0.195658
area_se                 0.057655
radius_worst            0.022023
perimeter_worst         0.019607
area_worst              0.017220
concave points_worst    0.048246
dtype: float64
   diagnosis  radius_mean  perimeter_mean  area_mean  concavity_mean  \
0          1     1.883367        1.792066   1.901098        0.520120   
1          1     1.702242        1.704696   1.699114        1.657269   
2          1     1.826893        1.856692   1.855464        1.660850   
3          1    -0.369407       -0.233678  -0.366714        1.373431   
4          1     1.380925        1.368953   1.391714        0.897723   

   concave points_mean   area_se  radius_worst  perimeter_worst  area_worst  \
0             1.029124  1.472536      1.808828         1.665246    1.845484   
1             1.862681  1.70789

In [78]:
#sacling
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['radius_mean', 'perimeter_mean', 'area_mean', 'concavity_mean',
       'concave points_mean', 'area_se', 'radius_worst', 'perimeter_worst',
       'area_worst', 'concave points_worst']
new_data=scaler.fit_transform(new_data[cols])
new_data=pd.DataFrame(new_data,columns=cols)
#added the target colm
new_data["diagnosis"]=y
print(new_data.head())

   radius_mean  perimeter_mean  area_mean  concavity_mean  \
0     0.914786        0.909368   0.917186        0.589538   
1     0.883663        0.894179   0.882802        0.883617   
2     0.905082        0.920603   0.909418        0.884543   
3     0.527687        0.557205   0.531128        0.810213   
4     0.828450        0.835813   0.830472        0.687190   

   concave points_mean   area_se  radius_worst  perimeter_worst  area_worst  \
0             0.743481  0.864440      0.936745         0.892013    0.945692   
1             0.950118  0.912050      0.904079         0.869617    0.908138   
2             0.887798  0.912872      0.878229         0.868514    0.884646   
3             0.797710  0.598586      0.626573         0.625380    0.637402   
4             0.763962  0.792467      0.886978         0.872178    0.890308   

   concave points_worst  diagnosis  
0              0.796708          1  
1              0.941435          1  
2              0.726986          1  
3         

In [79]:
x_new=new_data.drop("diagnosis",axis=1)
y_new=new_data["diagnosis"].astype(int) # Ensure y_new is of integer type

In [80]:
#train data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_new,y_new,test_size=0.2)
print(x_train.head())
print(x_test.head())

     radius_mean  perimeter_mean  area_mean  concavity_mean  \
450     0.592146        0.602427   0.589052        0.348734   
76      0.591539        0.594866   0.591926        0.333469   
37      0.574309        0.589223   0.576939        0.718765   
473     0.407680        0.408886   0.416398        0.146319   
201     0.609482        0.612186   0.616040        0.380901   

     concave points_mean   area_se  radius_worst  perimeter_worst  area_worst  \
450             0.443559  0.406753      0.620675         0.615071    0.626369   
76              0.402930  0.573607      0.608139         0.584065    0.615640   
37              0.760838  0.560268      0.636237         0.621285    0.646101   
473             0.139167  0.659372      0.489435         0.469299    0.487699   
201             0.351878  0.714477      0.603542         0.587107    0.619155   

     concave points_worst  
450              0.662905  
76               0.417565  
37               0.858444  
473              0.190

In [81]:
x_new.columns

Index(['radius_mean', 'perimeter_mean', 'area_mean', 'concavity_mean',
       'concave points_mean', 'area_se', 'radius_worst', 'perimeter_worst',
       'area_worst', 'concave points_worst'],
      dtype='object')

In [82]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=5)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [83]:
zn=model.predict([[21.67,132.86,1254.73,0.18453,23.54,0.657293,149.06,2018.0,0.1753,0.1598]])
print("Predicted data is:",zn)

Predicted data is: [1]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [84]:
from sklearn.metrics import accuracy_score
# Calculate accuracy on the test set
print(accuracy_score(y_test, y_pred))

0.9387755102040817
